In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

arquivo = '/content/DSFinal_global_cancer_patients/dataset/global_cancer_patients_2015_2024.csv'

df = pd.read_csv(arquivo)

#Limpeza de valores nulos, duplicados e exclusao da coluna ID
df = df.dropna()
df = df.drop_duplicates()
df = df.drop(columns=['Patient_ID'])

'''
ESSA PARTE AQUI PRA BAIXO PODE TIRAR
'''

#Verificacao de valores unicos
print("\nUnique Values for each non-numeric column:\n")
df_temp = df.select_dtypes(exclude=['number'])
df_temp['Year'] = df['Year']
for c in df_temp.columns:
    print(f"{c}:")
    print(df[c].unique())
    print('\n')

In [ ]:
# Remocao de valores irreais para propria descricao do dataset
# Pessoa não pode ter sobrevivido mais do que sua idade
df = df[df["Survival_Years"] <= df["Age"]]

In [ ]:
#Conversao de valores textuais para numericos
# Uso de One-Hot Encoding para variaveis nominais e map para variavel ordinal em sequencia

#Estagio do cancer
cancer_stage_map = {
    'Stage 0': 0,
    'Stage I': 1,
    'Stage II': 2,
    'Stage III': 3,
    'Stage IV': 4
}

df = pd.get_dummies(df, columns=[
    'Gender', 'Country_Region', 'Cancer_Type'], drop_first=True)
df['Cancer_Stage'] = df['Cancer_Stage'].map(cancer_stage_map)

In [ ]:
# Visualizacao dos dados numeros na busca por outliers
df_numeric_cols = df.select_dtypes(include=['int64', 'float64'])
df_numeric_cols = df_numeric_cols.drop(columns=['Year', 'Cancer_Stage'])

'''
ESSA EXIBICAO DE GRAFICOS AQUI TAMBEM PODE TIRAR
USEI PRA VISUALIZAR OS OUTLIERS
Se quiser deixar tambem fica bala
'''
for col in df_numeric_cols:
    plt.figure(figsize=(8, 1.5))
    sns.boxplot(data=df, x=col)
    plt.title(f'{col} Boxplot')
    plt.show()

In [ ]:
# Remocao dos outliers baseado no IQR
def clear_outliers(c):
    Q1 = df[c].quantile(0.25)
    Q3 = df[c].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    df_limpo = df[(df[c] >= lower_limit) & (df[c] <= upper_limit)]

    print(f"{df.shape[0] - df_limpo.shape[0]} Outlier values removed from '{c}'")
    return df_limpo

In [ ]:
for c in df_numeric_cols:
    df = clear_outliers(c)

In [ ]:
# Normalizacao dos dados com MixMaxScaler
# Transforma dados numeros entre valores de 0 a 1
scaler = MinMaxScaler()

for c in df_numeric_cols:
    df[c] = scaler.fit_transform(df[[c]])

df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_temp = df.select_dtypes(include=['int64', 'float64'])

correlation_matrix = df_temp.corr(numeric_only=True)

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Correlação")
plt.show()